# Week 04 Assignment weather data

Welcome to week four of this course programming 1. You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231.csv`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below.


<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. a slider widget selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from station 270. Select only the mean, minimum and maximum temperature. The data should look something like this:


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("/home/tijs/programming1/Programming1/data/KNMI_20181231.txt.tsv",
                 sep=",", 
                 usecols=[0, 1, 2, 3, 4], 
                 names=["station", "Date", "Tmean", "Tmin", "Tmax"], 
                 comment="#")
df = df[df.station == 270]
df

/home/tijs/DSLSvenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,station,Date,Tmean,Tmin,Tmax
97641,270,20000101,42,-4,79
97642,270,20000102,55,33,74
97643,270,20000103,74,49,89
97644,270,20000104,46,22,75
97645,270,20000105,41,14,56
...,...,...,...,...,...
104576,270,20181227,57,53,62
104577,270,20181228,71,58,81
104578,270,20181229,85,69,102
104579,270,20181230,80,68,90


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [3]:
#replace cells with spaces to NaN
df = df.replace(r"^\s*$", np.nan, regex=True)
#change data formats
df.Date = pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')
#change temperatures to celcius degrees
df.Tmean = pd.to_numeric(df.Tmean, errors='coerce') / 10
df.Tmin = pd.to_numeric(df.Tmin, errors='coerce') / 10
df.Tmax = pd.to_numeric(df.Tmax, errors='coerce') / 10
#remove leap year
df = df[~((df.Date.dt.month == 2) & (df.Date.dt.day == 29))]

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [4]:
#Test your outcome
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6935 entries, 97641 to 104580
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  6935 non-null   int64         
 1   Date     6935 non-null   datetime64[ns]
 2   Tmean    6935 non-null   float64       
 3   Tmin     6935 non-null   float64       
 4   Tmax     6935 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 325.1 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [5]:
def month_day(df_multipleyears):
    #your code to reform data here
    df_multipleyears_Tmin =  df_multipleyears.groupby([df_multipleyears.index.month, df_multipleyears.index.day]).Tmin.min()
    df_multipleyears_Tmax =  df_multipleyears.groupby([df_multipleyears.index.month, df_multipleyears.index.day]).Tmax.max()
    df_groupedbymonthday = pd.concat([df_multipleyears_Tmin,df_multipleyears_Tmax], axis=1)
    print(df_groupedbymonthday)
    return df_groupedbymonthday

In [6]:
#Test your code
def test_reformed(df):
    df_outside2018 = df[(df.Date.dt.year > 2007) & (df.Date.dt.year < 2018)]
    df_outside2018 = df_outside2018.reset_index().set_index(df_outside2018.Date)
    df_2018 = df[df.Date.dt.year == 2018]
    return month_day(df_outside2018), df_2018
    

df_outside2018, df_2018 = test_reformed(df)

           Tmin  Tmax
Date Date            
1    1     -5.8  11.1
     2     -7.5  10.2
     3    -12.6  10.7
     4     -4.1   9.2
     5     -6.0   8.4
...         ...   ...
12   27    -4.8  11.7
     28    -4.2  10.9
     29    -6.7  11.4
     30   -10.2   9.5
     31   -10.6  11.8

[365 rows x 2 columns]


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm


In [25]:
#your code here
gaussian = lambda x: np.exp(-0.5 * x * x)
rectangular = lambda x: (np.abs(x) <= 1.0).astype(np.float)
tricubic = lambda x: np.maximum((1.0 - abs(x)**3)**3, 0.0)
parabolic = lambda x: np.maximum(1.0 - x**2, 0.0)

def kernel_smooth(xs, xdata, ydata, width=1.0, window=gaussian):
    """Adapted from Datascience 1.
    Kernel smoothing method that fits a curve
    through data {xdata,ydata}, using a given
    window function with a specified width,
    evaluated at arguments xs.
    xs     = arguments x
    xdata  = {x0,x1,...,xn}
    ydata  = {y0,y1,...,yn}
    window = window function (default: gaussian)
    width  = window width (default: 1.0)
    """
    ys = []
    for x in xs:
        wdata = window((x - xdata) / width)
        y = np.sum(wdata * ydata)/ np.sum(wdata)
        ys.append(y)
    return np.array(ys)
#your motivation here
"""I would argue that kernel smoothing, also referred to as a rolling mean, is a simple yet
effective way to smooth this data. The point of smoothing this data is to show a upward and
downwards trend in temperature. I don't think a strict smoothing algorithm is necessary showing
the exact temperature between days."""

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



In [30]:
# smoothed_data = smooth_data(df.Date.dt.Days, )
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()

def make_plot(df_2018, df_outside2018):
    p = figure(title="Weather data", 
               background_fill_color="#fafafa")
    
    x2018 = df_2018.Date.dt.date
    x = [pd.Period(xi, freq='D').dayofyear for xi in x2018]
    y2018 = df_2018.Tmean
    p.line(x, y2018, color="blue", legend_label="2018")
    
    ymin = df_outside2018.Tmin
    ymax = df_outside2018.Tmax
    p.line(x, ymin, color="orange", legend_label="minimum outside 2018")
    p.line(x, ymax, color="hotpink", legend_label="maximum outside 2018")
    
    xs=np.linspace(1, 365, 365)
    y2018_smooth = kernel_smooth(xs, x, y2018, width=7, window=gaussian)
    p.line(x, y2018_smooth, color="green", width=2, legend_label="Smooth 2018")
    
    p.x_range=Range1d(0, 365)
    p.xaxis.axis_label = "days"
    p.yaxis.axis_label = "Temperature in degrees celcius"
    p.legend.label_text_font_size = "10px"
    p.legend.background_fill_alpha = 0.5
    p.grid.grid_line_color="white"
    return p
show(make_plot(df_2018, df_outside2018))

Loading BokehJS ...

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather